In [1]:
import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn import set_config
set_config(display="diagram")

In [4]:
data = pd.read_csv("../data/preprocessed.csv", index_col=0)
data.head()

,Rating,MRP,Processor,RAM,OS,Storage,Brand,OS_arch,RAM_Type,Processor_brand
0,4.2,25490.0,Celeron Dual Core,4,Windows 11,512 GB SSD,HP,64,LPDDR4,Intel
1,3.8,22490.0,Core i5,4,Windows 11,512 GB SSD,HP,64,LPDDR4X,Intel
2,4.1,25685.0,Celeron Dual Core,8,Windows 11,256 GB SSD,Lenovo,64,DDR4,Intel
3,4.4,51990.0,Core i5,8,Windows 11,512 GB SSD,ASUS,64,DDR4,Intel
4,4.2,38990.0,Core i3,8,Windows 11,512 GB SSD,ASUS,64,DDR4,Intel


In [72]:
data.drop(["Rating", "Brand"], axis=1, inplace=True)

In [73]:
X, y = data.drop("MRP", axis=1), data.MRP

In [96]:
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2)

In [97]:

pipeline = Pipeline([
    ("col_transformer_ohe", ColumnTransformer([("ohe", OneHotEncoder(handle_unknown="ignore"), [0,1,2,3,4,5,6])], remainder="passthrough")),
    ("col_transformer_sc", ColumnTransformer([("sc", StandardScaler(with_mean=False), [0,1,2,3,4,5,6])], remainder="passthrough")),
    ("estimator", LinearRegression())
])
pipeline.fit(xtrain, ytrain)

Pipeline(steps=[('col_transformer_ohe',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  [0, 1, 2, 3, 4, 5, 6])])),
                ('col_transformer_sc',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('sc',
                                                  StandardScaler(with_mean=False),
                                                  [0, 1, 2, 3, 4, 5, 6])])),
                ('estimator', LinearRegression())])

In [98]:
pipeline.score(xtest, ytest)

0.8637295326401773

In [114]:
sample = [["Core i5", 16, "Windows 11", "512 GB SSD", 64, "DDR4", "Intel"]]
int(pipeline.predict(sample))

/mnt/win10/D/virtual-env/ml_env/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


82752

In [99]:
joblib.dump(pipeline, "../model/model.obj")

['model/model.obj']